
# Install Libraries and Download Data 
---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/heart.csv")
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


# Feature Engineering 

In [ ]:
#Droping values that aren't necessary for comparing to smart watch data 
#This set will be used to examine modulating influence of excersize 
data = data.drop(columns=['age','sex','cp','fbs','ca','thal'])
data 

#This will be used for the models that predict for the values coming directly from smart watch 
data2 = data.drop(columns=['trestbps','chol','restecg','oldpeak','slope'])
data2 

,thalach,exang,target
0,168,0,0
1,155,1,0
2,125,1,0
3,161,0,0
4,106,0,0
...,...,...,...
1020,164,1,1
1021,141,1,0
1022,118,1,0
1023,159,0,1


Run one of these first to determine which data to build models off of 

In [ ]:
y = data['target']
X = data.drop('target',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [ ]:
y = data2['target']
X = data2.drop('target',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

Natural Bayes

In [ ]:
m2 = 'Naive Bayes'
nb = GaussianNB()
nb.fit(X_train,y_train)
nbpred = nb.predict(X_test)
nb_conf_matrix = confusion_matrix(y_test, nbpred)
nb_acc_score = accuracy_score(y_test, nbpred)
print("confussion matrix")
print(nb_conf_matrix)
print("\n")
print("Accuracy of Naive Bayes model:",nb_acc_score*100,'\n')
print(classification_report(y_test,nbpred))

confussion matrix
[[63 35]
 [18 89]]


Accuracy of Naive Bayes model: 74.14634146341463 

              precision    recall  f1-score   support

           0       0.78      0.64      0.70        98
           1       0.72      0.83      0.77       107

    accuracy                           0.74       205
   macro avg       0.75      0.74      0.74       205
weighted avg       0.75      0.74      0.74       205



RandomForest 

In [ ]:
m3 = 'Random Forest Classfier'
rf = RandomForestClassifier(n_estimators=20, random_state=12,max_depth=5)
rf.fit(X_train,y_train)
rf_predicted = rf.predict(X_test)
rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
rf_acc_score = accuracy_score(y_test, rf_predicted)
print("confussion matrix")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest:",rf_acc_score*100,'\n')
print(classification_report(y_test,rf_predicted))

confussion matrix
[[61 37]
 [11 96]]


Accuracy of Random Forest: 76.58536585365854 

              precision    recall  f1-score   support

           0       0.85      0.62      0.72        98
           1       0.72      0.90      0.80       107

    accuracy                           0.77       205
   macro avg       0.78      0.76      0.76       205
weighted avg       0.78      0.77      0.76       205



Support Vector Classifier

In [ ]:
m7 = 'Support Vector Classifier'
svc =  SVC(kernel='rbf', C=2)
svc.fit(X_train, y_train)
svc_predicted = svc.predict(X_test)
svc_conf_matrix = confusion_matrix(y_test, svc_predicted)
svc_acc_score = accuracy_score(y_test, svc_predicted)
print("confussion matrix")
print(svc_conf_matrix)
print("\n")
print("Accuracy of Support Vector Classifier:",svc_acc_score*100,'\n')
print(classification_report(y_test,svc_predicted))

confussion matrix
[[68 30]
 [27 80]]


Accuracy of Support Vector Classifier: 72.1951219512195 

              precision    recall  f1-score   support

           0       0.72      0.69      0.70        98
           1       0.73      0.75      0.74       107

    accuracy                           0.72       205
   macro avg       0.72      0.72      0.72       205
weighted avg       0.72      0.72      0.72       205



In [ ]:
#Testing a possible value from the smart watch 
X_test = [164,1]
y_pred = svc.predict([X_test])

print(y_pred)
if y_pred[0] == 0:
  print("There is no disease")
else:
  print("There is disease")

[1]
There is disease


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
